# Inflation Indexes and Curves 2 (Quantlib comparison)

This guide replicates and is a comparison to the Quantlib tutorial page at
https://www.quantlibguide.com/Inflation%20indexes%20and%20curves.html

## Inflation Indexes

Historical index fixings in *rateslib* should be indexed to the 1st of the appropriate inflation month.

In [ ]:
from rateslib import *
from pandas import Series, MultiIndex

In [ ]:
inflation_fixings = [
    (dt(2022, 1, 1), 110.70),
    (dt(2022, 2, 1), 111.74),
    (dt(2022, 3, 1), 114.46),
    (dt(2022, 4, 1), 115.11),
    (dt(2022, 5, 1), 116.07),
    (dt(2022, 6, 1), 117.01),
    (dt(2022, 7, 1), 117.14),
    (dt(2022, 8, 1), 117.85),
    (dt(2022, 9, 1), 119.26),
    (dt(2022, 10, 1), 121.03),
    (dt(2022, 11, 1), 120.95),
    (dt(2022, 12, 1), 120.52),
    (dt(2023, 1, 1), 120.27),
    (dt(2023, 2, 1), 121.24),
    (dt(2023, 3, 1), 122.34),
    (dt(2023, 4, 1), 123.12),
    (dt(2023, 5, 1), 123.15),
    (dt(2023, 6, 1), 123.47),
    (dt(2023, 7, 1), 123.36),
    (dt(2023, 8, 1), 124.03),
    (dt(2023, 9, 1), 124.43),
    (dt(2023, 10, 1), 124.54),
    (dt(2023, 11, 1), 123.85),
    (dt(2023, 12, 1), 124.05),
    (dt(2024, 1, 1), 123.60),
    (dt(2024, 2, 1), 124.37),
    (dt(2024, 3, 1), 125.31),
    (dt(2024, 4, 1), 126.05),
]
dates, values = zip(*inflation_fixings)
cpi_fixings = Series(values, dates)

*Rateslib* contains an `index_value` method that will determine such for a given reference value date and other common parameters.

In [ ]:
index_value(
    index_lag=0,
    index_method="monthly",
    index_fixings=cpi_fixings,
    index_date=dt(2024, 3, 15)
)

For example to replicate the *Quantlib* example of a lagged reference date we can use:

In [ ]:
index_value(
    index_lag=3,
    index_method="daily",
    index_fixings=cpi_fixings,
    index_date=dt(2024, 5, 15)
)

## Inflation Curves

Create a nominal discount curve for cashflows. Calibrated to a 3% continuously compounded rate.

In [ ]:
nominal_curve = Curve(
    nodes={dt(2024, 5, 11): 1.0, dt(2074, 5, 18): 1.0},
    interpolation="log_linear",
    convention="Act365F",
    id="discount"
)
solver1 = Solver(
    curves=[nominal_curve],
    instruments=[Value(dt(2074, 5, 11), metric="cc_zero_rate", curves="discount")],
    s=[3.0],
    id="rates",
    instrument_labels=["nominal"],
)

Now create an inflation curve, based on the last known CPI print, calibrated with zero coupon inflation swaps rates. Notice that an inflation curve starts as of the last known fixing as its `index_base`. This is similar to *Quantlib*, not be design, but by necessity since this is the only information we have that can define the start of the curve.

In [ ]:
inflation_curve = Curve(
    nodes={
        dt(2024, 4, 1): 1.0,  # <- last known inflation print.
        dt(2025, 5, 11): 1.0,  # 1y
        dt(2026, 5, 11): 1.0,  # 2y
        dt(2027, 5, 11): 1.0,  # 3y
        dt(2028, 5, 11): 1.0,  # 4y
        dt(2029, 5, 11): 1.0,  # 5y
        dt(2031, 5, 11): 1.0,  # 7y
        dt(2034, 5, 11): 1.0,  # 10y
        dt(2036, 5, 11): 1.0,  # 12y
        dt(2039, 5, 11): 1.0,  # 15y
        dt(2044, 5, 11): 1.0,  # 20y
        dt(2049, 5, 11): 1.0,  # 25y
        dt(2054, 5, 11): 1.0,  # 30y
        dt(2064, 5, 11): 1.0,  # 40y
        dt(2074, 5, 11): 1.0,  # 50y
    },
    interpolation="log_linear",
    convention="Act365F",
    index_base=126.05,
    index_lag=0,
    id="inflation"
)
solver = Solver(
    pre_solvers=[solver1],
    curves=[inflation_curve],
    instruments=[
        ZCIS(dt(2024, 5, 11), "1y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "2y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "3y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "4y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "5y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "7y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "10y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "12y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "15y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "20y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "25y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "30y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "40y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "50y", spec="eur_zcis", curves=["inflation", "discount"], leg2_index_fixings=cpi_fixings),
    ],
    s=[2.93, 2.95, 2.965, 2.98, 3.0, 3.06, 3.175, 3.243, 3.293, 3.338, 3.348, 3.348, 3.308, 3.228],
    instrument_labels=["1y", "2y", "3y", "4y", "5y", "7y", "10y", "12y", "15", "20y", "25y", "30y", "40y", "50y"],
    id="zcis",
)       

The data can be output to a table or plotted as below.

In [ ]:
inflation_curve.plot("1m")

In [ ]:
inflation_curve.plot_index(right=dt(2030, 6, 1))

Some of the forecast values from the curve can be obtained directly from *Curve* methods.

In [ ]:
inflation_curve.index_value(dt(2027, 4, 1), index_lag=3, index_method="monthly")

In [ ]:
inflation_curve.index_value(dt(2027, 4, 1), index_lag=0, index_method="monthly")

In [ ]:
inflation_curve.index_value(dt(2027, 5, 15), index_lag=3, index_method="daily")

## Seasonality

The way *rateslib* handles seasonality is to replicate it via its *CompositeCurve* framework. With this approach, adding seasonality can be done in a multitude of ways, but since Quantlib has used *seasonaility factors* we will synthesize that approach within *rateslib's* framework. We create a new curve with **node dates** reflecting the key dates used by Quantlib.

The theory here for *rateslib* is as follows.

A discount factor on a *rateslib* *CompositeCurve* is very approximately the product of its contained curves (c1 and c2):

$$
v_{cc} \approx v_{c1} v_{c2}
$$

The *CompositeCurve* index value takes its *index base* from the first (main) curve in its composition, therefore:

$$
I_{value(cc)} = \frac{I_{base(c1)}}{v_{cc}} \approx = \frac{I_{base(c1)}}{v_{c1}v_{c2}} = I_{value(c1)} \frac{1}{v_{c2}}
$$

If we set the *index base* on the seasonality curve (c2) **exactly to 1.0** this equation reduces to:

$$
I_{value(cc)} \approx I_{value(c1)} I_{value(c2)}
$$

So the index value on the *CompositeCurve* equals the product of the two index values.

Thus, if we set the set the *index values* on the seasonality curve (c2) directly they can act directly as multipliers to the underlying inflation values on the inflation curve.
This seasonality curve can be calibrated using a *Solver*, and in this particular case (to match Quantlib) they are calibrated directly with *index values*.

Here we add the same seasonality factors in years 2025, 2026 and 2027.

In [ ]:
season = Curve(
    nodes={
        dt(2024, 4, 1): 1.0,
        dt(2024, 12, 1): 1.0,
        **{dt(2025, _, 1): 1.0 for _ in range(1, 13)},
        **{dt(2026, _, 1): 1.0 for _ in range(1, 13)},
        **{dt(2027, _, 1): 1.0 for _ in range(1, 13)},
        dt(2074, 5, 11): 1.0,
    },
    interpolation="log_linear",
    convention="Act365F",
    index_base=1.0,  #  <- as per theory above
    index_lag=0,  # <- matches the main inflation curve
    id="seasonality"
)

The seasonality factors will be added by directly inserting index values at specific dates.

In [ ]:
multipliers = [
    1.003245,
    1.001994,
    0.999715,
    1.000495,
    1.000929,
    0.998687,
    0.995949,
    0.994682,
    0.995949,
    1.000519,
    1.003705,
    1.004186,
]
season_solver = Solver(
    curves=[season],
    instruments=\
        [Value(dt(2024, 12, 1), curves=[season], metric="index_value")] +\
        [Value(dt(2025, _, 1), curves=[season], metric="index_value") for _ in range(1, 13)] +\
        [Value(dt(2026, _, 1), curves=[season], metric="index_value") for _ in range(1, 13)] +\
        [Value(dt(2027, _, 1), curves=[season], metric="index_value") for _ in range(1, 13)],
    s=[1.0] + multipliers*3
)

In [ ]:
season.plot("1b", left=dt(2024, 4, 1), right=dt(2028, 4, 1))

Once the seasonality is designed it must be composited with an underlying inflation curve and the instruments re-solved

In [ ]:
inflation_with_season = CompositeCurve([inflation_curve, season], id="inflation_s")

In [ ]:
solver = Solver(
    pre_solvers=[solver1],  # nominal discount curve
    curves=[inflation_with_season, inflation_curve],
    instruments=[
        ZCIS(dt(2024, 5, 11), "1y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "2y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "3y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "4y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "5y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "7y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "10y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "12y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "15y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "20y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "25y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "30y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "40y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
        ZCIS(dt(2024, 5, 11), "50y", spec="eur_zcis", curves=["inflation_s", "discount"], leg2_index_fixings=cpi_fixings),
    ],
    s=[2.93, 2.95, 2.965, 2.98, 3.0, 3.06, 3.175, 3.243, 3.293, 3.338, 3.348, 3.348, 3.308, 3.228],
    instrument_labels=["1y", "2y", "3y", "4y", "5y", "7y", "10y", "12y", "15y", "20y", "25y", "30y", "40y", "50y"],
    id="zcis",
)

In [ ]:
inflation_with_season.plot("1b", comparators=[inflation_curve], left=dt(2024, 9, 1), right=dt(2030, 9, 1))

In [ ]:
inflation_with_season.plot_index(comparators=[inflation_curve], left=dt(2024, 9, 1), right=dt(2030, 9, 1))

### New sampled values

In [ ]:
inflation_with_season.index_value(dt(2027, 4, 1), index_lag=3, index_method="monthly")

In [ ]:
inflation_with_season.index_value(dt(2027, 4, 1), index_lag=0, index_method="monthly")

In [ ]:
inflation_with_season.index_value(dt(2027, 5, 15), index_lag=3, index_method="daily")

The trick here is obviously to find a representation of a `seasonality` curve that matches one's expectation of seasonality adjustments. Here, a `Solver` calibration was used to separately solve the `seasonality` curve to inflation rate adjustments.

In [ ]:
for i, date in enumerate([dt(2025, _, 1) for _ in range(1, 13)]+[dt(2026, _, 1) for _ in range(1, 13)]):
    a = inflation_curve.index_value(date, index_lag=0, index_method="monthly")
    b = inflation_with_season.index_value(date, index_lag=0, index_method="monthly")
    print(f"Date: {date}, Inflation curve: {float(a):.5f}, With seasonality: {float(b):.5f}, Multiplier: {float(b/a):.7f}, Intended Multiplier: {multipliers[i%12]:.7f}")

## Inflation Swap and DV01

We can easily construct a ZCIS or other type of inflation based *instrument* and use the native `delta` and `gamma` methods associated with a `Solver` to extract risk sensitivities.

In [ ]:
zcis = ZCIS(dt(2024, 3, 11), "4y", spec="eur_zcis", curves=["inflation_s", "discount"], fixed_rate=3.0, leg2_index_fixings=cpi_fixings)

In [ ]:
zcis.rate(solver=solver)

In [ ]:
zcis.npv(solver=solver)

In [ ]:
df = zcis.delta(solver=solver)
descriptors = df.agg(["sum"])
descriptors.index = MultiIndex.from_tuples([("sum", "sum", "sum")])
df.style.format(precision=0).concat(descriptors.style.format(precision=0))

This risk display shows no exposure to the **seasonality** configuration becuase the *Solver* that was used to calibrate the seasonality was not added as a ``pre_solver`` to the *Solver* used in the last step. Therefore it treats the seasonality as a **constant** and not as variable to which risk sensitivities can be obtained.

To observe the change, simply replace ``pre_solvers=[solver1]`` with ``pre_solvers=[solver1, season_solver]``, in the above *Solver* and re run the cells.

In [ ]:
zcis.gamma(solver=solver).style.format(precision=1)